In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from functools import reduce
import json
from IPython.display import display, Markdown

In [2]:
sample_submission=pd.read_csv("../data/raw/sample_submission.csv")
specs=pd.read_csv("../data/raw/specs.csv")
test=pd.read_csv("../data/raw/test.csv")
train=pd.read_csv("../data/interim/train.csv")
train_labels=pd.read_csv("../data/raw/train_labels.csv")

Those games roughly correspond to assessments which should be predicted. Looks like activities are given not for learning, but for relaxation (like fireworks).

In [3]:
train.title[train.type=="Game"].unique()

array(['Scrub-A-Dub', 'All Star Sorting', 'Air Show', 'Crystals Rule',
       'Dino Drink', 'Bubble Bath', 'Dino Dive', 'Chow Time',
       'Pan Balance', 'Happy Camel', 'Leaf Leader'], dtype=object)

In [4]:
train.title[train.type=="Assessment"].unique()

array(['Mushroom Sorter (Assessment)', 'Bird Measurer (Assessment)',
       'Cauldron Filler (Assessment)', 'Cart Balancer (Assessment)',
       'Chest Sorter (Assessment)'], dtype=object)

In [6]:
pd.options.display.max_colwidth=999
games_description=pd.DataFrame(data=[
    {"game": "Chow Time", "target": "balance the scales", 
    "fields": ["scale_weight", "target_weight", "resources", "scale_contents"]},
    {"game": "All Star Sorting", "target": "sort dinosaurs by houses",
    "fields": ["dinosaurs", "houses", "dinosaurs_placed"]},
    {"game": "Dino Dive", "target": "select dinosaur(s) which raises water in pool to given level (memory + measure)", 
     "fields":["target_level", "target_water_level"]},
    {"game": "Scrub-A-Dub", "target": "find item with right size for animal several times", 
     "fields": ["round_target", "size"]},
    {"game": "Pan Balance", "target": "Balance weights",
     "fields": ["target_weight", "weights"]},
    {"game": "Dino Drink", "target": "match sizes of eggs with holes",
    "fields": ["holes"]},
    {"game": "Air Show", "target": "Move dinousaur N spaces (into target_distances)", 
    "fields": ["distance", "target_distances"]},
    {"game": "Bubble Bath", "target": "Find container with right size", 
    "fields": ["target_containers", "containers"]},
    {"game": "Crystals Rule", "target": "Measure things",
    "fields": ["group", "target_size"]},
    {"game": "Happy Camel", "target": "Find bowl with toy using scales", 
     "fields": ["bowl_id", "scale_contents"]},
    {"game": "Leaf Leader", "target": "Balance scales",
     "fields": ["target_weight", "weight"]},
    
])
game_count=train[train.type=="Game"].groupby("title")["game_session"].nunique().reset_index(name="n_games")
game_count.columns=["game", "n_games"]
pd.merge(games_description, game_count, on="game").sort_values("n_games", ascending=False)

,game,target,fields,n_games
0,Chow Time,balance the scales,"[scale_weight, target_weight, resources, scale_contents]",5804
3,Scrub-A-Dub,find item with right size for animal several times,"[round_target, size]",4202
1,All Star Sorting,sort dinosaurs by houses,"[dinosaurs, houses, dinosaurs_placed]",3819
5,Dino Drink,match sizes of eggs with holes,[holes],3418
7,Bubble Bath,Find container with right size,"[target_containers, containers]",3165
9,Happy Camel,Find bowl with toy using scales,"[bowl_id, scale_contents]",3128
2,Dino Dive,select dinosaur(s) which raises water in pool to given level (memory + measure),"[target_level, target_water_level]",3066
6,Air Show,Move dinousaur N spaces (into target_distances),"[distance, target_distances]",2876
10,Leaf Leader,Balance scales,"[target_weight, weight]",2831
4,Pan Balance,Balance weights,"[target_weight, weights]",2806


In [6]:
assessment_description=pd.DataFrame(data=[
    {"game": "Mushroom Sorter (Assessment)", "target": "Sort 3 mushrooms",
     "fields": ["stumps"]},
    {"game": "Bird Measurer (Assessment)", "target": "Measure 3 birds height",
     "fields": ["bird_height", "caterpillar", "caterpillars"]},
    {"game": "Cauldron Filler (Assessment)", "target": "Match buckets with cauldrons, then choose right bucket",
     "fields": ["bucket", "cauldron", "target_bucket"]},
    {"game": "Cart Balancer (Assessment)", "target": "Balance scales", 
     "fields": ["crystals", "left", "right"]},
    {"game": "Chest Sorter (Assessment)", "target": "Order chests using pan balance",
     "fields": ["left", "right", "pillars"]}
])
assessment_description

,game,target,fields
0,Mushroom Sorter (Assessment),Sort 3 mushrooms,[stumps]
1,Bird Measurer (Assessment),Measure 3 birds height,"[bird_height, caterpillar, caterpillars]"
2,Cauldron Filler (Assessment),"Match buckets with cauldrons, then choose right bucket","[bucket, cauldron, target_bucket]"
3,Cart Balancer (Assessment),Balance scales,"[crystals, left, right]"
4,Chest Sorter (Assessment),Order chests using pan balance,"[left, right, pillars]"


In [7]:
def show_game_session(df, title, filter_cols=tuple()):
    random_row = df[df.title == title].sample(1)
    installation_id = random_row.installation_id.iloc[0]
    session_id = random_row.game_session.iloc[0]
    show_player_sessions(df, installation_id, session_id, filter_cols)


def show_player_sessions(df, installation_id, session_id=None, filter_cols=tuple()):
    df = df[df.installation_id == installation_id].copy()
    game_sessions = [session_id] if session_id else df.game_session.unique()
    for game_session in game_sessions:
        session = df[df.game_session==game_session].copy()
        display(Markdown(describe_session(game_session, session)))
        with pd.option_context('display.max_rows', 999, 'display.max_columns', 999, 
                               'display.max_colwidth', 999):
            display(format_session(session, filter_cols))


def describe_session(game_session, session):
    head = session.sort_values("event_count").head(1)
    if "version" in json.loads(head.event_data.iloc[0]):
        version = json.loads(head.event_data.iloc[0])["version"]
    else:
        version = "-"
    return f"__{game_session} {head.title.iloc[0]} {head.type.iloc[0]} {head.world.iloc[0]} v{version}__ "


def format_session(session, filter_cols):
    session = session.sort_values("event_count")
    session["timestamp"]=session["timestamp"].str[:19]
    session["event_data"] = session.event_data.apply(json.loads)
    columns = sorted(reduce(set.__or__, session.event_data.apply(lambda d: set(d.keys())).values))
    for c in columns:
        session[c] = session.event_data.apply(lambda x: x.get(c, '-'))
    session["version"]=None
    session = session.drop(["game_session", "installation_id", "event_data", "game_time", 
                            "title", "type", "world", "version"] + [c for c in filter_cols if c in session.columns], axis=1)
    session.set_index("event_count", inplace=True)
    
    return session


#for game in set(train.title[train.type=="Game"].unique()):
#    show_game_session(train, game, ["coordinates"])

In [8]:
#for game in set(test.title[test.type=="Assessment"].unique())-set(assessment_description.game.values):
#    show_game_session(train, game, ["coordinates"])

In [10]:
show_game_session(train, "Chow Time", ["coordinates"])

__324567f2a2a0be36 Chow Time Game CRYSTALCAVES v1.0__ 

,Unnamed: 1,event_id,timestamp,event_code,index,correct,description,diet,dinosaur,duration,identifier,media_type,misses,resources,round,scale_contents,scale_weight,source,target_weight,total_duration,weight
event_count,,,,,,,,,,,,,,,,,,,,,
1,6367,7d093bf9,2019-08-27T17:07:19,2000,9505030.0,-,-,-,-,-,-,-,-,-,0,-,-,-,-,-,-
2,6368,f93fc684,2019-08-27T17:07:22,4010,9505031.0,-,-,-,-,-,-,-,-,-,0,-,-,-,-,-,-
3,6369,7ec0c298,2019-08-27T17:07:25,3010,9505032.0,-,"It's Chow Time! We have some VERY hungry dinosaurs waiting. Let's feed them! Big dinosaurs eat a lot of food, small dinosaurs only eat a little!",-,-,-,"1431,1401b",animation,-,-,0,-,-,-,-,10333,-
4,6370,0d1da71f,2019-08-27T17:07:35,3110,9505033.0,-,"It's Chow Time! We have some VERY hungry dinosaurs waiting. Let's feed them! Big dinosaurs eat a lot of food, small dinosaurs only eat a little!",-,-,11367,"1431,1401b",animation,-,-,0,-,-,-,-,-,-
5,6371,63f13dd7,2019-08-27T17:07:35,2020,9505034.0,-,-,herbivore,arnie,-,-,-,-,"[3, 1, 3, 1]",1,-,-,-,8,-,-
6,6372,7ec0c298,2019-08-27T17:07:35,3010,9505035.0,-,"To make the sure dinosaur eats the right amount tap and drag the food to the scale, until it's balanced! The scale is balanced when the two arrows are lined up, and green You can add, or take away, food if the scale is not balanced.",-,-,-,"1434,1435REVA,3200",animation,-,-,1,-,-,-,-,16000,-
7,6373,0d1da71f,2019-08-27T17:07:50,3110,9505036.0,-,"To make the sure dinosaur eats the right amount tap and drag the food to the scale, until it's balanced! The scale is balanced when the two arrows are lined up, and green You can add, or take away, food if the scale is not balanced.",-,-,14713,"1434,1435REVA,3200",animation,-,-,1,-,-,-,-,-,-
8,6374,cfbd47c8,2019-08-27T17:07:52,4030,9505037.0,-,-,-,-,-,-,-,-,"[1, 3, 1]",1,-,-,resources,-,-,3
9,6375,4ef8cdd3,2019-08-27T17:07:52,4020,9505038.0,True,-,-,-,835,-,-,-,-,1,[3],3,-,-,-,3
